In [ ]:
import torch
from torchvision import datasets
from torch import nn
from torchvision.transforms import ToTensor
import numpy as np

# 读取数据

In [ ]:
batch_size = 256
train_data = datasets.FashionMNIST(
    root='../fasionMNIST/data', 
    train=True, 
    download=False, 
    transform=ToTensor()
    )
test_data = datasets.FashionMNIST(
    root='../fasionMNIST/data',
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size)

# 定义模型

In [72]:
num_inputs = 784
num_outputs = 10

class LinearNet(nn.Module):
    def __init__(self, num_inpus, num_outputs):
        super(LinearNet, self).__init__() 
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x):
        logits = self.linear(x.view(x.shape[0], -1))
        return logits
    
model = LinearNet(num_inputs, num_outputs)
for param in model.parameters():
    print(param.shape)

# 初始化权重和偏置
nn.init.normal_(model.linear.weight, mean=0, std=0.01)
nn.init.constant_(model.linear.bias, val=0)

torch.Size([10, 784])
torch.Size([10])


Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# 定义损失函数和优化器

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# 分别对weight和bias设置优化器来设置正则化
# optimizer_w = torch.optim.SGD(model.linear.weight, lr=0.1, weight_decay=0.1)
# optimizer_b = torch.optim.SGD(model.linear.bias, lr=0.1)

# 训练模型

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X, y in dataloader:
        pred =  model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss, correct = 0, 0
    for X, y in dataloader:
        pred = model(X)
        loss += loss_fn(pred, y)
        correct += (pred.argmax(dim=1) == y).float().sum().item()
    loss /= num_batches
    correct /= size
    print("Test Error: {:.6f}\nAccuracy: {:.2f}%".format(loss, 100*correct))

num_epochs = 10
for epoch in range(num_epochs):
    print("Epoch {}:\n------------".format(epoch))
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
        